# Лабораторная работа №1-2 по СМАД

## Тема: генерация экспериментальных данных по схеме имитационного моделирования на ЭВМ

---

## Вариант 2

Информация о моделируемом объекте:

-   число факторов - 2
-   По первому фактору зависимость близка к линейной (возрастающей)
-   По второму фактору зависимость близка к параболической
-   Максимальное значение отклика приходится на граничные точки точки области действия факторов

## Задание:

### Часть 1:

1. Определить модель, область действия факторов, план эксперимента
2. Написать программу по генерации экспериментальных данных
3. Построить графики зависимости незашумленного отклика от входных факторов

### Часть 2:

1. Сформировать программные модули по вычислению МНК-оценок параметров для заданной параметрической модели
2. Оценить параметры модели объекта
3. Проверить адекватность полученной модели.


Импортируем нужные библиотеки


In [ ]:
import numpy as np
from prettytable import PrettyTable
import matplotlib.pyplot as plt


Задаем класс Модель

Параметры модели $\theta$ = [1, 4, 0.001, 4]


In [ ]:
class Model:

    def __init__(self):
        self.amount_tests = 12
        self.x_max = 1
        self.x_min = -1
        self.x1 = []
        self.x2 = []
        self.signal = []
        self.response = []
        self.power = 0
        self.variance = 0
        self.theta = [1, 4, 0.001, 4]
        self.func = lambda x1, x2: 1 + 4*x1 + 0.001 * x1 + 4 * x2 ** 2


In [ ]:
class Calculator:

    @staticmethod
    def compute_signal(model: Model):
        signal = [model.func(model.x1[i], model.x2[i])
                  for i in range(model.amount_tests)]
        return np.array(signal)

    @staticmethod
    def compute_variance(model) -> float:
        return (model.power * 0.1)

    @staticmethod
    def compute_power(model):
        avg_signal = [
            np.sum(model.signal) / len(model.signal)
            for i in range(len(model.signal))
        ]
        vec_avg_signal = np.array(avg_signal)
        power = np.vdot(model.signal - vec_avg_signal,
                        model.signal - vec_avg_signal) / len(model.signal)
        return power

    @staticmethod
    def compute_response(model, error):
        return (model.signal + error)


Создаем модуль генерации случайных значений

Первая функция генерирует пару независимых переменных.
Вторая функция генерирует величину ошибки с нормальным распределением и нулевым мат. ожиданием


In [ ]:
class DataGenerator:

    @staticmethod
    def generate_couple(x_min, x_max, amount_tests):
        x1 = np.random.uniform(x_min, x_max, amount_tests)
        x2 = np.random.uniform(x_min, x_max, amount_tests)
        return x1, x2

    @staticmethod
    def generate_error(standart_deviation, number_tests) -> float:
        error = np.random.normal(0, standart_deviation, number_tests)
        return error


Заполняем модель данными


In [ ]:
model = Model()  # инициализируем модель

x1, x2 = DataGenerator.generate_couple(
    model.x_min, model.x_max, model.amount_tests)

model.x1, model.x2 = x1, x2

model.signal = Calculator.compute_signal(model)
model.power = Calculator.compute_power(model)
model.variance = Calculator.compute_variance(model)

error = DataGenerator.generate_error(
    model.variance, model.amount_tests)

model.response = Calculator.compute_response(model, error)


Отрисовываем таблицу выборки и параметры модели


In [ ]:
table = PrettyTable()
table.field_names = ["Num Test", "x1", "x2", "signal", "error", "response"]
for i in range(model.amount_tests):
    table.add_row([i+1, np.around(model.x1[i], 7),
                   np.around(model.x2[i], 7),
                   np.around(model.signal[i], 7),
                   np.around(error[i], 7),
                   np.around(model.response[i], 7)])
print(table)
print("\n")
print(f"Параметры модели Θ: {model.theta}")
print(f"Мощность сигнала: {model.power}")
print(f"Дисперсия σ: {model.variance}")


Построение графиков зависимости незашумленного отклика от входных параметров


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
fig.set_figheight(6)
fig.set_figwidth(12)
ax_x1 = np.linspace(-1, 1, 1000)
ax_x2 = ax_x1
axes[0].set(title="η(x1, 0)")
axes[1].set(title="η(x2, 0)")
axes[0].plot(ax_x1, model.func(ax_x1, 0))
axes[1].plot(ax_x2, model.func(0, ax_x2))
axes[0].grid()
axes[1].grid()
plt.show()
